# This is a script to develop the extraction of the data from funda.nl

The functions are written 


In [14]:
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup
import re
import glob

In [63]:
# Read the file properly 

# define the file path 
file_path = r'C:\Users\tosam\OneDrive\Desktop\Coding\Funda_Scraping\funda-scraper\funda_scraper\source.txt'


# define a function to read the file   
def read_file(file_path):
    '''
    This function reads the HTML source code
    '''
    
    with open(file_path, 'r', errors = 'ignore') as file:
        html_content = file.read()
        
    return html_content    
    
#def extract_occurance():
#    '''
#    To extract the data based on the keywords 
#    '''
    
# define a function to extract numbers from the string
'''
def num_extract(text):
    
    # converts the , to . from dutch numericals 
    #test_str = text.replace(',', '.')
    numbers = 0
    num_ = []
    
    numbers = re.findall(r'\d+\.\d+', text)
     
    if len(numbers)> 0:  
        # convert to actual numbers
        for i in numbers:
            num_.append(i.replace(".", ""))
        
    # Then check if there is a dutch ','
    elif len(numbers) == 0:
        numbers = re.findall(r'\d+\,\d+',text)
        
        if len(numbers) > 0:
    #    # convert to actual numbers
            for i in numbers:
                num_.append(i.replace(",", "."))

    # if there is no decimal point 
    if len(numbers) == 0:
        num_.append(re.findall(r'\d+', text)[0])
    
    return np.array(num_, dtype = np.float32)    
'''    
# extract the phrase using this function   
# extract all the content from 'kenmerken' onwards until

def text_extract( text, phrase, start, end ):
    '''
    This function extracts text based on keywords....
    params:
        text: from which the string has to be extracted
        Phrase: to indicate the point of extract
        Start:  starting value from the poistion of the phrase
        end:    to specify the end value 
    '''
    val = 0
    A = [i for i in range(len(text)) if text.startswith(phrase, i)]
    
    if len(A)==1:
        val = text[A[0]-start:A[0]+end]
    
        return A, val
    else:
        val = 'detected more than once or none'
        return np.nan, val

# extract numbers    
def num_extract(text):
    '''
    To extract numbers to check for integral or decimal numbers.
    
    '''
    
    # remove the comma 
    text = text.replace(',', '')
    numbers = 0
    num_ = 0
    
    # find all numbers with decimal
    numbers = re.findall(r'\d+\.\d+', text)
     
    if len(numbers)==0:
        numbers = re.findall(r'\d+', text)
         
        
        if len(numbers)==0:
            return np.nan
        
        else:
            num_ = numbers
            return np.array(num_, dtype = float)[0]
    else:
        num_ = numbers
        return np.array(num_, dtype = float)[0]
    
    

In [175]:
filename_list = glob.glob('E:\Scraping\Scraping_Data\**.txt')
file_path = filename_list[5]
html_text  = read_file(file_path)

In [176]:
# extract the HTML text
html_text  = read_file(file_path)
# extract the imprtant chubnk and remove the garbage values 

inx_start = text_extract( html_text, r'>Features<', 0,  100)
inx_end = text_extract( html_text, r'>Popularity<', 0,  100)

# keep only the useful part of the text
useful_text = html_text[inx_start[0][0]: inx_end[0][0]]

# Now, we extract everythig  based on the keywords

#.......... for the fraag prijs
refine_1 = text_extract(useful_text, r'>Asking price<', 0, 1000)
#search the position of the euro sign
refine_2 = text_extract(refine_1[1], u"\N{euro sign}", 0, 10)
if refine_2[0] is np.nan:
    refine_2 = text_extract(refine_1[1], "â", 0, 10)
# extract the the number 
fraag_prijs = num_extract(refine_2[1])

# .......... for Vraagprijs per m2
refine_1 = text_extract(useful_text, r'>Asking price per ', 0, 1000)
#search the position of the euro sign
refine_2 = text_extract(refine_1[1], u"\N{euro sign}", 0, 10)
if refine_2[0] is np.nan:
    refine_2 = text_extract(refine_1[1], "â", 0, 10)
# extract the the number 
fraag_prijs_per_m2 = num_extract(refine_2[1])

# .......... for Bouwjaar
refine_1 = text_extract(useful_text, r'>Year of construction<', 0, 600)
A = [i for i in range(len(refine_1[1])) if refine_1[1].startswith(r'<span class="comment">',i)]
if len(A) ==0:
    year = np.nan
else:   
    year = num_extract(refine_1[1][A[0]:A[1]])

# .......... for Wonen
refine_1 = text_extract(useful_text, r'Living area', 0, 600)
refine_2 = text_extract(refine_1[1], 'm\u00b2', 5, 5)
if refine_2[0] is np.nan:
    refine_2 = text_extract(refine_1[1], 'mÂ', 5, 5)
area_living = num_extract(refine_2[1])
area_living

# .......... for parceel
refine_1 = text_extract(useful_text, r' Plot ', 0, 600)
if refine_1[0] is np.nan:
    plot_area = np.nan
else:
    refine_2 = text_extract(refine_1[1], 'm\u00b2', 5, 5)
    if refine_2[0] is np.nan:
        refine_2 = text_extract(refine_1[1], 'mÂ', 5, 5)
    plot_area = num_extract(refine_2[1])
plot_area

# .......... for Inhoud
refine_1 = text_extract(useful_text, r'>Volume in cubic meters<', 0, 600)
if refine_1[0] is np.nan:
    plot_vol = np.nan
else:
    refine_2 = text_extract(refine_1[1], 'm\u00b3', 5, 5)
    if refine_2[0] is np.nan:
        refine_2 = text_extract(refine_1[1], 'mÂ', 5, 5)
plot_vol = num_extract(refine_2[1])
plot_vol

# .......... for Externe bergruimte
refine_1 = text_extract(useful_text, r'>External storage space<', 0, 600)
if refine_1[0] is np.nan:
    external_storage = np.nan
else:
    refine_2 = text_extract(refine_1[1], 'm\u00b2', 5, 5)
    if refine_2[0] is np.nan:
        refine_2 = text_extract(refine_1[1], 'mÂ', 5, 5)
    external_storage = num_extract(refine_2[1])
external_storage 

# .......... for Aantal kamers
refine_1 = text_extract(useful_text, r'>Number of rooms<', -len('Number of rooms '), 600)
refine_2 = text_extract(refine_1[1], r' rooms ', 5,0)
total_rooms = num_extract(refine_2[1])
total_rooms

# ........... for slaapkamers
refine_3 = text_extract(refine_1[1], r'bedroom', 4,12)
bedrooms = num_extract(refine_3[1])

# ........... for bad kamers
refine_1 = text_extract(useful_text, r'>Number of bath', -len('Number of bathroom '), 800)
refine_2 = text_extract(refine_1[1], r' bathroom', 4,4)
total_bathrooms = num_extract(refine_2[1])
total_bathrooms

refine_3 = text_extract(refine_1[1], r'separate', 4,4)
if refine_3[0] is np.nan:
    sep_toilet = 0
else:
    sep_toilet = num_extract(refine_3[1])
sep_toilet 


# ........... for woonlagen 
refine_1 = text_extract(useful_text, r'>Number of stories<', -len('Number of stories '), 600)
if refine_1[0] is np.nan:
    floors = 0
    attic = np.nan
else:
    refine_2 = text_extract(refine_1[1], 'stor', 4, 4) # for singular and plural values
    floors = num_extract(refine_2[1])

    # ............for attic .....
    refine_2 = text_extract(refine_1[1], 'stor', 0, 30)

    if 'attic' in refine_2[1]:
        attic = True
    else:    
        attic = False
        
# ........... for energie label

refine_1 = text_extract(useful_text, '>Energy label<', 0, 2000)
energy_label_list = ['A+++++','A++++', 'A+++', 'A++', 'A+', 'A', 'B', 'C', 'D', 'E', 'F', 'G'] 

for i in energy_label_list:
    refine_2 = text_extract(refine_1[1], r'>'+i+'<', 4, 4)
    
    if refine_2[0] is np.nan:
        continue
    else:
        energy_label = i
        break
energy_label

'B'